In [ ]:
# default_exp measures

# Behavioural Measures

> Compute behavioural measures from raw transaction data

In [1]:
#hide
from nbdev.showdoc import *

A behavioural measure is a number derived from a...